In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.optim as optim
import torch.nn as nn

train_df = pd.read_csv("train.csv").astype(float)
test_df = pd.read_csv("test.csv").astype(float)
train_data = train_df.values
test_data = test_df.values

trainX_array = train_data[:, :-1]
trainY_array = train_data[:, -1].reshape(-1, 1)

testX_np = torch.from_numpy(test_data)
testY_np = torch.from_numpy(test_data)

trainX_float16 = trainX_array.astype(np.float16)
trainX_float64 = trainX_float16.astype(np.float64)
abs_diff = torch.abs(torch.from_numpy(trainX_array) - trainX_float64)
max_abs_diff = torch.max(abs_diff).item()

coef = torch.tensor([
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],  # POS_50_59_CP
    [0.0300],  # POS_60_69_CP
    [0.0300],
    [0.0300],
    [0.0300]
], dtype=torch.float64)

testX_np = testX_np.to(dtype=torch.float64)[:, :-1]

first_row = testX_np[0]
predicted_deaths = (first_row @ coef).item()

predicted_deaths1 = (testX_np @ coef)
mean = torch.mean(predicted_deaths1).item()

num_features = trainX_array.shape[1]
coef = torch.zeros((num_features, 1), dtype=torch.float64)

predictions = torch.from_numpy(trainX_array) @ coef
mse = torch.mean((predictions - trainY_array)**2)

torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(torch.from_numpy(trainX_array), torch.from_numpy(trainY_array))
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

num_features = trainX_array.shape[1]
coef = torch.zeros((num_features, 1), dtype=torch.float64, requires_grad=True)
loss_fn = nn.MSELoss()

optimizer = optim.SGD([coef], lr=0.000002)

num_epochs = 500
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch_X, batch_Y in dl:
        predictions = batch_X @ coef
        loss = loss_fn(predictions, batch_Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dl)
    if epoch % 100 == 0:
        train_predictions = torch.from_numpy(trainX_array) @ coef
        train_mse = loss_fn(train_predictions, torch.from_numpy(trainY_array))

In [2]:
#Q1: 
element_size = trainX_array.itemsize  
num_elements = trainX_array.size 
bytes_consumed = element_size * num_elements
bytes_consumed

83520

In [3]:
#Q2: 
float(max_abs_diff) 

0.0

In [4]:
#Q3:
torch.cuda.is_available()

False

In [5]:
#Q4: 
predicted_deaths

9.844

In [6]:
#Q5: 
mean 

12.073632183908048

In [7]:
#Q6
x = torch.tensor(0.0)
y = x ** 2 - 8 * x + 19
ans = float(y)
ans

19.0

In [8]:
#Q7
import torch.optim as optim
def function_to_minimize(x):
    return x**2 - 8*x + 19
x = torch.tensor(0.0, requires_grad=True)
optimizer = optim.SGD([x], lr=0.1)  
num_iterations = 1000
for i in range(num_iterations):
    optimizer.zero_grad()
    y = function_to_minimize(x)
    y.backward()  
    optimizer.step() 

x_minimized = x.item()
x_minimized

3.999999523162842

In [9]:
#Q8 
mse.item()

197.8007662835249

In [10]:
#Q9
train_mse.item()

26.564613484128618

In [11]:
#Q10
import pandas as pd
import torch

train_df = pd.read_csv("train.csv").astype(float)
test_df = pd.read_csv("test.csv").astype(float)

train_data = torch.tensor(train_df.values, dtype=torch.float64)
test_data = torch.tensor(test_df.values, dtype=torch.float64)

trainX = train_data[:, :-1]
trainY = train_data[:, -1].reshape(-1, 1)
testX = test_data[:, :-1]
testY = test_data[:, -1].reshape(-1, 1)

ds = torch.utils.data.TensorDataset(trainX, trainY)

test_predictions = testX @ coef 

test_mse = loss_fn(test_predictions, testY)
test_mse.item()


29.05854692548551